# Chapter 7. 앙상블 학습과 랜덤 포레스트

### 앙상블 방법 <sup>ensemble method</sup>
* 앙상블 학습 알고리즘
* 대중의 지혜와 비슷하게 일련의 예측기(즉, 분류나 회귀 모델)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있다.
* 일련의 예측기를 **앙상블**부르기 때문에, 이를 **앙상블 학습**이라고 한다.
* 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 일련의 결정 트리 분류기를 훈련시킬 수 있다. 모든 개별 트리의 예측을 구하고 그런 다음 가장 많은 선택을 받은 클래스를 예측으로 삼는다.

### 랜덤 포레스트 <sup>ramdom forest</sup>
* 결정 트리의 앙상블
* 간단한 방법임에도 오늘날 강력한 머신러닝 알고리즘 중 하나

## Set up

In [21]:
import numpy as np

## 7.1 투표 기반 분류기

다수결 투표로 정해지는 분류기를 **직접 투표**<sup>hard voting</sup> 분류기라고 한다.

큰 수의 법칙<sup>law of large numbers</sup>에 따라 각 분류기가 약한 학습기<sup>weak learner</sup> (즉, 랜덤 추측보다 조금 더 높은 성능을 내는 분류기) 일지라도 충분하게 많고 다양하다면 앙상블은 (높은 정확도를 내는) 강한 학습기<sup>strong learner</sup> 가 될 수 있다.

앙상블 방법은 예측기가 가능한 한 서로 독립적일 때 최고의 성능을 발휘한다. 다양한 분류기를 얻는 한 가지 방법은 각기 다른 알고리즘으로 학습시키는 것이다. 이렇게 하면 매우 다른 종류의 오차를 만들 가능성이 높기 때문에 앙상블 모델의 정확도를 향상시킨다.

In [1]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting = 'hard'
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [2]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.888
SVC 0.896
VotingClassifier 0.904


### 간접 투표 <sup>soft voting</sup>
* 모든 분류기가 클래스의 확률을 예측할 수 있으면 (즉, predict_proba() 메서드가 있으면) 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있다.
* 확률이 높은 투표에 비중을 더 두기 때문에 직접 투표 방식보다 성능이 높다.
* voting = "soft" 옵션을 사용하고 모든 분류기가 클래스의 확률을 추정할 수 있어야 한다.
 * SVG는 기본값으로 클래스 확률을 제공하지 않으ㅡ므로 probability 매개변수를 True로 지정해야 한다. (이때 교차 검증을 사용하므로 훈련 속도가 느려진다) 

In [3]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability = True)

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting = 'soft'
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

In [4]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.904


## 7.2 배깅과 페이스팅

다양한 분류기를 만드는 또 다른 방법은 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시키는 것이다.

* 배깅<sup>bagging</sup>
 * 훈련 세트에서 중복을 허용하여 샘플링하는 방식
* 페이스팅<sup>pasting</sup>
 * 중복을 허용하지 않고 샘플링하는 방식
 
수집 함수는 전형적으로 분류일 때 **통계적 최빈값**<sup>statistical mode</sup> (즉, 직접 투표 분류기처럼 가장 많은 예측 결과)이고 회귀에 대해서는 평균을 계산한다.

개별 예측기는 크게 편향되더라도 수집 함수를 통과하면 편향과 분산이 모두 감소한다. 일반적으로 앙상블 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬 때와 비교해 편향은 비슷하지만 분산은 줄어든다.

예측기는 모두 동시에 다른 CPU 코어나 서버에서 병렬로 학습시킬 수 있다. 이와 유사하게 예측도 병렬로 수행할 수 있다. (확장성)

### 7.2.1 사이킷런의 배깅과 페이스팅

In [5]:
# 회귀는 BaggingRegressor API 사용
# BaggingClassifier는 기반이 되는 분류기가 클래스 확률을 추정할 수 있으면 직접 투표 대신 자동으로 간접 투표 방식을 사용한다
from sklearn.ensemble import BaggingClassifier 
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), # 결정 트리 분류기 사용
    n_estimators = 500, # 분류기 500개
    max_samples = 100, # 무작위로 선택된 100개의 샘플
    bootstrap = True, # 배깅은 True, 페이스팅은 False
    n_jobs = -1 # 훈련과 예측에 사용할 CPU 코어 수. -1은 가용한 모든 코어 사용
)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로 배깅이 페이스팅보다 편향이 조금 높다. 하지만 다양성은 예측기들의 상관 관계를 줄이므로 앙상블의 분산을 감소시킨다. 전반적으로 배깅이 더 나은 모델을 만들기 때문에 일반적으로 더 선호하는 경향이 있다.

시간과 CPU 파워에 여유가 있다면 교차 검증으로 배깅과 페이스팅을 모두 평가하여 더 나은쪽을 선택하는 것이 좋다.

### 7.2.2 oob 평가

배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있다. BaggingClassifier는 평균적으로 각 예측기에 63% 정도만 샘플링된다. 이때 선택되지 않은 훈련 샘플의 나머지 37%를 oob<sup>out-of-bag</sup> 샘플이라고 부른다. 예측기마다 oob 샘플은 모두 다르다.

앙상블의 평가는 별도의 검증 세트 필요 없이 각 예측기의 oob 평가를 평균하여 얻는다.

In [6]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators = 500,
    max_samples = 100,
    bootstrap = True,
    n_jobs = -1,
    oob_score = True # 훈련이 끝난 후 자동으로 oob 평가를 수행
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.92

In [7]:
from sklearn.metrics import accuracy_score

y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.928

In [8]:
# oob 샘플에 대한 결정 함수의 값 확인
# 결정 함수는 각 훈련 샘플의 클래스 확률을 반환한다 (기반이 되는 예측기가 predict_proba() 메서드를 가지고 있기 때문)
# 음성 클래스에 속할 확률, 양성 클래스에 속할 확률
bag_clf.oob_decision_function_

array([[0.37817259, 0.62182741],
       [0.42819843, 0.57180157],
       [1.        , 0.        ],
       [0.0125    , 0.9875    ],
       [0.02486188, 0.97513812],
       [0.0959596 , 0.9040404 ],
       [0.38441558, 0.61558442],
       [0.06994819, 0.93005181],
       [0.94910941, 0.05089059],
       [0.81233933, 0.18766067],
       [0.56153846, 0.43846154],
       [0.04065041, 0.95934959],
       [0.78552279, 0.21447721],
       [0.8680203 , 0.1319797 ],
       [0.88502674, 0.11497326],
       [0.1023622 , 0.8976378 ],
       [0.05333333, 0.94666667],
       [0.92245989, 0.07754011],
       [0.66052632, 0.33947368],
       [0.96514745, 0.03485255],
       [0.05066667, 0.94933333],
       [0.19791667, 0.80208333],
       [0.90414508, 0.09585492],
       [0.9870801 , 0.0129199 ],
       [0.96850394, 0.03149606],
       [0.00268097, 0.99731903],
       [0.95739348, 0.04260652],
       [1.        , 0.        ],
       [0.03684211, 0.96315789],
       [0.73536896, 0.26463104],
       [0.

## 7.3 랜덤 패치와 랜덤 서브스페이스

BaggingClassifier는 특성 샘플링도 지원한다. 샘플링은 `max_features`, `bootstrap_features` 두 매개변수로 조절된다. 특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춘다.

이 기법은 (이미지와 같은) 매우 고차원의 데이터셋을 다룰 때 유용하다.

* 랜덤 패치 방식<sup>random patches method</sup>
 * 훈련 특성과 샘플을 모두 샘플링한다.
* 랜덤 서브스페이스 방식<sup>random subspaces method</sup>
 * 훈련 샘플은 모두 사용 (bootstrap = False && max_samples = 1.0)
 * 특성은 샘플링 (bootstrap_features = True || max_features < 1.0)


## 7.4 랜덤 포레스트

* 랜덤 포레스트
 * 일반적으로 배깅(혹은 페이스팅)을 적용한 결정 트리의 앙상블
 * 전형적으로 max_samples를 훈련 세트의 크기로 지정한다
 * `DecisionTreeClassifier` 대신 결정 트리에 최적화되어 사용하기 편리한 `RandomForestClassifier` 사용 가능
 * 마찬가지로 회귀에는 `RandomForestRegressor` 사용

In [9]:
# RandomForestClassifier는 몇 가지 예외가 존재하나 
# (트리 성장을 조절하기 위한) DecisionTreeClassifier 매개변수와 
# (앙상블 자체를 제어하는데 필요한) BaggingClassifier 매개변수 모두 가지고 있다
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(
    n_estimators = 500, # 500개의 트리
    max_leaf_nodes = 16, # 최대 16개의 리프 노드
    n_jobs = -1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [11]:
# 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 
# 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입한다
# 이는 트리를 더욱 다양하게 만들고 편향을 손해보는 대신 분산을 낮추어 전체적으로 더 훌륭한 모델을 만든다.

# BaggingClassifier로 RandomForestClassifier 유사하게 만듦:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features = "auto", max_leaf_nodes = 16),
    n_estimators = 500, max_samples = 1.0, bootstrap = True, n_jobs = -1)

### 7.4.1 엑스트라 트리

#### 엑스트라 트리 <sup>extra-trees</sup>
* 익스트림 랜덤 트리 <sup>extremely randomized trees</sup>
* 극단적으로 무작위한 트리의 랜덤 포레스트
 * 최적의 임곗값을 찾는 대신 후보 특성을 이용해 무작위로 분할한 다음 그중에서 최상의 분할 선택
* 분류에는 `ExtraTreesClassifier`, 회귀에는 `ExtraTreesRegressor` 사용
* 일반적으로 랜덤 포레스트, 엑스트라 트리 둘다 시도해보고 교차 검증으로 비교해보는 것이 유일한 방법
 * 그리드 탐색으로 하이퍼파라미터 튜닝을 한다

### 7.4.2 특성 중요도

랜덤 포레스트의 또 다른 장점은 특성의 상대적 중요도를 측정하기 쉽다는 것.
사이킷런은 어떤 특성을 사용한 노드가 (모든 트리에 걸쳐) 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도를 측정한다. 가중치 평균이며 각 노드의 가중치는 연관된 훈련 샘플 수와 같다.

In [12]:
from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf = RandomForestClassifier( n_estimators = 500, n_jobs = -1 )
rnd_clf.fit(iris["data"], iris["target"])

# feature_importances_: 사이킷런은 훈련이 끝난 뒤 특성마다 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결괏값을 정규화한다.
# iris 데이터셋 특성 별 중요도 출력
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09785640024079285
sepal width (cm) 0.020992419525793487
petal length (cm) 0.4144898771668969
petal width (cm) 0.4666613030665168


## 7.5 부스팅

* 부스팅<sup>boosting</sup>
 * 가설 부스팅<sup>hypothesis boosting</sup>
 * 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법
 * 가장 인기 있는 것은 **에이다부스트**<sup>AdaBoost(adaptive boosting)</sup>와 **그레이디언트 부스팅**<sup>gradient boosting</sup>

### 7.5.1 에이다부스트

* 이전 모델이 과소적합했던(잘못 분류된) 훈련 샘플의 가중치를 더 높이는 것
* 샘플의 가중치를 업데이트하면서 순차적으로 학습
* 연속된 학습 기법에는 중요한 단점이 있다. 확장성이 높지 않다는 것.
 * 각 예측기는 이전 예측기가 훈련되고 평가된 후에 학습될 수 있기 때문에 병렬화(혹은 분할)를 할 수 없다.
* 분류에는 `AdaBoostClassifier`, 회귀에는 `AdaBoostRegressor` 사용


In [14]:
from sklearn.ensemble import AdaBoostClassifier

# 클래스가 두 개뿐일 때 SAMME가 에이다부스트와 동일 (예측기 가중치 식 동일)
# 예측기가 클래스 확률을 추정할 수 있다면 SAMME.R (Real) 이라는 변종 사용
# SAMME.R은 예측값 대신 클래스 확률에 기반하며 일반적으로 성능이 더 좋다
# 에이다부스트 앙상블이 훈련 세트에 과대적합되면 추정기 수를 줄이거나 추정기의 규제를 더 강하게 한다
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = 1), # 결정 노드 1개, 리프 노드 2개
    n_estimators = 200, # 200개의 결정 트리
    algorithm = "SAMME.R", # 에이다부스트 다중클래스
    learning_rate = 0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 7.5.2 그레이디언트 부스팅

* 에이다부스트처럼 앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가한다
* 그러나 반복마다 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 **잔여 오차**<sup>residual error</sup>에 새로운 예측기를 학습시킨다

#### 그레이디언트 트리 부스팅 <sup>gradient tree boosting</sup>
* 그레이디언트 부스티드 회귀 트리<sup>gradieent boosted regression tree (GRBT)</sup>

In [23]:
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [24]:
from sklearn.tree import DecisionTreeRegressor

# 잡음이 섞인 2차 곡선 형태의 훈련 세트
tree_reg1 = DecisionTreeRegressor(max_depth = 2, random_state=42)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [25]:
y2 = y - tree_reg1.predict(X) # 잔여 오차
tree_reg2 = DecisionTreeRegressor(max_depth = 2, random_state=42)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [26]:
y3 = y2 - tree_reg2.predict(X) # 잔여 오차
tree_reg3 = DecisionTreeRegressor(max_depth = 2, random_state=42)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [27]:
# 모든 트리의 예측을 더하여 새로운 샘플에 대한 예측을 만듦 (앙상블 모델)
X_new = np.array([[0.8]])
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [28]:
y_pred

array([0.75026781])

In [30]:
from sklearn.ensemble import GradientBoostingRegressor

# RandomForestRegressor 와 비슷하게 앙상블 훈련 제어하는 매개변수 존재: max_depth, min_samples_leaf
# learning_rate 각 트리의 기여 정도를 조절
# 축소(shrinkage) 
# learning_rate = 0.1 낮게 설정하면 앙상블을 훈련 세트에 학습 시키기 위해 많은 트리가 필요하다
# but 일반적으로 예측의 성능은 좋아진다
gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 3, learning_rate = 1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [33]:
# 많은 수의 트리를 먼저 훈련 시키고 최적의 수를 찾는 방법

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 120)
gbrt.fit(X_train, y_train)

# 최적의 트리 수를 찾기 위한 조기 종료 기법 (4장 참조)
# 훈련의 각 단계에서 앙상블에 의해 만들어진 예측기를 순회하는 반복자 반환
errors = [mean_squared_error(y_val, y_pred)
             for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1 # 최적의 트리 수

gbrt_best = GradientBoostingRegressor(max_depth = 2, n_estimators = bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=108)

In [35]:
# 실제로 훈련을 중지하는 방법

# warmstart = True 사이킷런이 fit() 메서드가 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 함
gbrt = GradientBoostingRegressor(max_depth = 2, warm_start = True)

min_val_error = float("inf")
error_going_up = 0

# 연속해서 5번의 반복 동안 검증 오차가 향상되지 않으면 훈련을 멈춤
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break # 조기 종료

#### 확률적 그레이디언트 부스팅 <sup>stochastic gradient boosting</sup>
* `subsample` 매개변수로 각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정
 * subsample = 0.25 면 각 트리는 무작위로 선택된 25% 훈련 샘플로 학습
* 편향이 높아지는 대신 분산이 낮아진다
* 훈련 속도를 상당히 높인다

#### XGBoost
* 익스트림 그레이디언트 부스팅 <sup>extreme gradient boosting</sup>
* 최적화된 그레이디언트 부스팅 구현으로 유명한 파이썬 라이브러리
* 패키지 목표는 매우 빠른 속도, 확장성, 이식성
* 자동 조기 종료와 같은 여러 좋은 기능도 제공

In [38]:
! pip3 install xgboost

     |████████████████████████████████| 1.2 MB 2.1 MB/s eta 0:00:01


In [40]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

XGBoostError: XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed (vcomp140.dll or libgomp-1.dll for Windows, libomp.dylib for Mac OSX, libgomp.so for Linux and other UNIX-like OSes). Mac OSX users: Run `brew install libomp` to install OpenMP runtime.
  * You are running 32-bit Python on a 64-bit OS
Error message(s): ['dlopen(/usr/local/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /usr/local/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: image not found']


In [41]:
xbg_reg.fit(X_train, y_train,
            eval_set=[(X_val, y_val)], 
            early_stopping_rounds = 2) # 자동 조기 종료
y_pred = xbg_reg.predict(X_val)

NameError: name 'xbg_reg' is not defined

## 7.6 스태킹

* 스태킹<sup>stacking(stacked generalization)</sup>
 * 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수(e.g. 직접 투표) 대신 취합하는 모델을 훈련시킨다
 * 예측기들은 각각 다른 값을 예측하고 마지막 예측기(**블렌더**<sup>blender</sup> 혹은 **메타 학습기**<sup>meta learner</sup>)가 각 예측 값을 입력으로 받아 최종 예측을 만듦
 * 블렌더를 학습시키는 일반적인 방법은 홀드 아웃<sup>hold-out</sup> 세트를 사용하는 것
 * 사이킷런은 스태킹을 직접 지원하지 않으므로 직접 구현하거나 DESlib 같은 오픈 소스를 사용한다